In [172]:
%matplotlib inline

from compare_bmp import compare_images
from PIL import Image, ImageFilter
from matplotlib.pyplot import imshow
import numpy as np

We'll start with this image:
![img1](images/original/image.bmp)

In [173]:
# let's get our message set up
message = list('this is a message')
# convert to binary representation
message = [bin(ord(x)) for x in message]
print(message)
# split the binary into 
message = [[bit for bit in x][2:] for x in message]
print(message)
# flatten it and convert to integers
message = [int(bit) for sublist in message for bit in sublist]
print(message)
originalMessage = message

['0b1110100', '0b1101000', '0b1101001', '0b1110011', '0b100000', '0b1101001', '0b1110011', '0b100000', '0b1100001', '0b100000', '0b1101101', '0b1100101', '0b1110011', '0b1110011', '0b1100001', '0b1100111', '0b1100101']
[['1', '1', '1', '0', '1', '0', '0'], ['1', '1', '0', '1', '0', '0', '0'], ['1', '1', '0', '1', '0', '0', '1'], ['1', '1', '1', '0', '0', '1', '1'], ['1', '0', '0', '0', '0', '0'], ['1', '1', '0', '1', '0', '0', '1'], ['1', '1', '1', '0', '0', '1', '1'], ['1', '0', '0', '0', '0', '0'], ['1', '1', '0', '0', '0', '0', '1'], ['1', '0', '0', '0', '0', '0'], ['1', '1', '0', '1', '1', '0', '1'], ['1', '1', '0', '0', '1', '0', '1'], ['1', '1', '1', '0', '0', '1', '1'], ['1', '1', '1', '0', '0', '1', '1'], ['1', '1', '0', '0', '0', '0', '1'], ['1', '1', '0', '0', '1', '1', '1'], ['1', '1', '0', '0', '1', '0', '1']]
[1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 

In [174]:
# first, open the original image and hide a message in it
imgpath = 'images/original/image.bmp'

img = Image.open(imgpath)

# we'll use simple repetition as a very rudimentary ECC to try to maintain integrity
# each bit of the message will be repeated 6 times - the two LSBs of the R,G, and B values of one pixel
#imgArray = np.copy(np.asarray(img))[:1]
imgArray = list(np.asarray(img))

def set_bit(val, bitNo, bit):
    """ given a value, which bit in the value to set, and the actual bit (0 or 1) to set, return the new value """
    mask = 1 << bitNo
    val &= ~mask
    if bit:
        val |= mask
    return val

msgIndex = 0
newImg = []

for row in imgArray:
    newRow = []
    for pixel in row:
        newPixel = []
        for val in pixel:
            # iterate through RGB values, one at a time
            #print(bin(r))
            if msgIndex >= len(message):
                setTo = 0
            else:
                setTo = message[msgIndex]
                msgIndex += 1
            val = set_bit(val, 0, setTo)
            val = set_bit(val, 1, setTo)
            newPixel.append(val)
        newRow.append(newPixel)
    newImg.append(newRow)

arr = np.array(newImg, np.uint8)
im = Image.fromarray(arr)
im.save("image_steg.bmp")

How does the image look now?

![image with hidden data](image_steg.bmp)

Awesome! Doesn't look like anything's wrong.

In [175]:
# now let's blur the image to destroy some data
blurredpath = 'image_blurred.bmp'

img = Image.open("image_steg.bmp")
blurred = img.copy().filter(ImageFilter.BLUR)

blurred.save(blurredpath)

And here it is now that we've blurred it:
![img_blurred](images/image_blurred.bmp)

In [176]:
# last, open the image and apply our ECC strategy to see if the message made it through

blurredImg = Image.open(blurredpath)

imgArray = list(np.asarray(blurredImg))

message = []

#for row in imgArray:
row = imgArray[0]
for pixel in row:
    count = {"0": 0, "1": 0}
    for val in pixel:
        # iterate through RGB values, one at a time
        #print(bin(r))
        for i in [-1, -2]:
            bit = int(bin(val)[i])
            if bit == 0:
                count["0"] += 1
            else:
                count["1"] += 1
        if count["1"] >= count["0"]:
            message.append(1)
        else:
            message.append(0)

In [177]:
from math import floor
# let's see what the message says!

outputMessage = []

originalMessage = list('this is a message')
originalMessage = [ord(x) for x in originalMessage]

for i in range(floor(len(message)/7)):
    start = i * 7
    # this is gross
    char = message[start:start+7]
    char = ''.join([str(c) for c in char])
    outputMessage.append(int(char, 2))

print(originalMessage)
print(outputMessage)

print("ORIGINAL VS. OUTPUT")
print("===================")
print(''.join([chr(c) for c in originalMessage]))
print(''.join([chr(c) for c in outputMessage]))

[116, 104, 105, 115, 32, 105, 115, 32, 97, 32, 109, 101, 115, 115, 97, 103, 101]
[116, 104, 56, 113, 97, 83, 119, 67, 71, 6, 111, 63, 15, 95, 7, 62, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ORIGINAL VS. OUTPUT
this is a message
th8qaSwCGo?_>                                                                                                                                                                                                     
